In [ ]:
import pandas as pd
import numpy as np
import mne

from functions.bad_channels import bad_channels
from _parameters import *

In [ ]:
def preprocess(s: int, file: str):
    
    # Read data
    ses1, ses2 = file

    raw1 = mne.io.read_raw_bdf(ses1, preload=True)
    raw2 = mne.io.read_raw_bdf(ses2, preload=True)

    raw = mne.concatenate_raws([raw1, raw2])

    # Re-reference
    raw.set_eeg_reference(channels['ref'])

    # Bipolar emg/eog
    raw = mne.set_bipolar_reference(raw, 'EXG3', 'EXG4', 'EOG',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG5', 'EXG6', 'EMGL',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG7', 'EXG8', 'EMGR',
                                    copy = False, drop_refs = True)

    raw.set_channel_types({'EOG':'eog'})
    raw.set_channel_types({'EMGL':'emg', 'EMGR':'emg'})

    # Drop and interpolate
    raw.drop_channels(channels['drop'])

    raw = bad_channels(raw, picks = "eeg", sig_lev=0.05)
    raw.interpolate_bads()

    # High pass filter EMG
    raw.filter(l_freq = 60, h_freq = None, picks = ['EMGL','EMGR'])

    # Resample
    raw.resample(200)

    # Montage
    biosemi64 = mne.channels.make_standard_montage('biosemi64')
    eeg_map = dict(zip(raw.info.ch_names[:64],biosemi64.ch_names))

    raw.rename_channels(eeg_map)
    raw.set_montage(biosemi64)

    # Save clean raw
    raw_fname = dirs['home'] + dirs['raw'] + '/raw_s' + str(s) + '.fif'
    
    raw.save(raw_fname, overwrite = True)

    return raw

In [ ]:
s = 1; file = files['eeg'][0]

raw = preprocess(s, file)

raw.info

In [ ]:
def run_ica(s: int, raw, auto_pick = False):

    # High-pass filter
    raw.filter(1,None)

    # Fit ICA
    ica = mne.preprocessing.ICA(n_components=.99, max_iter="auto").fit(raw)
    
    # EOG correlates
    ica.exclude, scores = ica.find_bads_eog(raw)

    if auto_pick: # Exclude max
        ica.plot_scores(scores)
        ica.plot_components(scores.argmax())
        ica.exclude = [int(scores.argmax())]

    else: # Pick components
        ica.plot_components()
        ica.plot_scores(scores)

        bad_comp = input('Components to exclude (_,_): ').split(sep=',')
        ica.exclude = [int(c) for c in bad_comp]

    # Save
    ica_fname = dirs['home'] + dirs['ica'] + '/ica_s' + str(s) + '.fif'
    ica.save(ica_fname, overwrite = True)

    return ica

In [ ]:
ica = run_ica(1, raw, auto_pick = True)